In [15]:
from configs import API_KEY, DEFAULT_MODEL
import os
import openai
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [2]:
os.environ["OPENAI_API_KEY"] = API_KEY
# openai.api_key = API_KEY
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
RunnablePassthrough().invoke(['hi'])

['hi']

In [4]:
chat_template_tools = ChatPromptTemplate.from_template("""What are the five most important tools a {job} needs?
    answer only by listing the tools.""")

chat_template_strategy = ChatPromptTemplate.from_template("""Considering the tools provided, 
    develop a strategy to effectively learn and master them: {tools}""")

In [5]:
chat_template_tools

ChatPromptTemplate(input_variables=['job'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job'], input_types={}, partial_variables={}, template='What are the five most important tools a {job} needs?\n    answer only by listing the tools.'), additional_kwargs={})])

In [6]:
chat = ChatOpenAI(
    model_name = 'gpt-4',
    model_kwargs = {'seed': 365},
    temperature = 0,
    max_tokens = 100
)

D:\Projects\pyspark\nlp_course\venv\Lib\site-packages\IPython\core\interactiveshell.py:3641: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [7]:
 string_parser = StrOutputParser()

In [8]:
chain_tools = chat_template_tools | chat | string_parser
chain_strategy = chat_template_strategy | chat | string_parser

In [9]:
print(chain_tools.invoke({'job': 'Data Scientist'}))

1. Python
2. R Programming Language
3. SQL (Structured Query Language)
4. Tableau
5. Hadoop


In [10]:
print(chain_strategy.invoke({'tools': """1. Python
2. R Programming
3. SQL
4. Tableau
5. Hadoop"""}))

1. Python: Start with the basics of Python, such as variables, data types, operators, and control flow. Once you have a good understanding of these, move on to more complex topics like functions, classes, and error handling. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding regularly on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming: Start with the basics of


## Runnable passthrough

In [11]:
chain_tools = chat_template_tools | chat | string_parser | {'tools' : RunnablePassthrough()}
chain_strategy = chat_template_strategy | chat | string_parser

In [12]:
chain_combined = chain_tools | chain_strategy

In [13]:
print(chain_combined.invoke({'job': 'Data Scientist'}))

1. Python: Start with the basics of Python, such as variables, data types, operators, and control flow. Once you have a good understanding of these, move on to more complex topics like functions, classes, and exceptions. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding regularly on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming: Start with the basics of R


In [14]:
chain_combined.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       

In [16]:
chat_template_books = ChatPromptTemplate.from_template("""Suggest three of the best intermediate level {programming_language} books
                        Answer only by listing the books.""")

chat_template_projects = ChatPromptTemplate.from_template("""Suggest three interesting {programming_language} projects suitable for 
                        intermediate level programmers.
                        Answer only by listing the projects.""")

In [17]:
chain_books = chat_template_books | chat | string_parser
chain_projects = chat_template_projects | chat | string_parser

In [18]:
chain_parallel = RunnableParallel({'books': chain_books, 'projects':chain_projects})

In [19]:
chain_parallel.invoke({'programming_language': 'python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Weather Forecasting Application using APIs\n2. Developing a Web Scraper for E-commerce Websites\n3. Creating a Personal Finance Tracker/Budgeting App'}

In [20]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [21]:
chat_template_time = ChatPromptTemplate.from_template(""" I am an intermediate level programmer.
        Consider the following literature : {books}
        Also, consider the following projects: {projects}
        Roughly how much time would it take to complete the literature and the projects?""")

In [22]:
chat = ChatOpenAI(
    model_name = 'gpt-4',
    model_kwargs = {'seed': 365},
    temperature = 0,
    max_tokens = 500
)

D:\Projects\pyspark\nlp_course\venv\Lib\site-packages\IPython\core\interactiveshell.py:3641: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [23]:
chain_time = (RunnableParallel({'books': chain_books, 'projects':chain_projects}) 
              | chat_template_time
              | chat
              | string_parser)

In [24]:
print(chain_time.invoke({'programming_language': 'python'}))

The time it takes to complete the literature and the projects can vary greatly depending on several factors such as your reading speed, comprehension level, the complexity of the projects, and the amount of time you can dedicate each day. 

However, as a rough estimate:

1. "Fluent Python: Clear, Concise, and Effective Programming" - This book is around 800 pages. If you read and practice for about 2 hours a day, it might take you around 1-2 months to complete.

2. "Python Cookbook: Recipes for Mastering Python 3" - This book is around 700 pages. Again, if you read and practice for about 2 hours a day, it might take you around 1-2 months to complete.

3. "Effective Python: 90 Specific Ways to Write Better Python" - This book is around 230 pages. If you read and practice for about 2 hours a day, it might take you around 2-3 weeks to complete.

For the projects:

1. Building a Web Scraper with BeautifulSoup - If you work on it for about 2-3 hours a day, it might take you around 1-2 weeks

In [25]:
chain_time.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

When multiple runnable objects are piped together with another runnable objects, we don't need to explicitly need to mention RunnableParallel

In [26]:
chain_time2 = (({'books': chain_books, 'projects':chain_projects}) 
              | chat_template_time
              | chat
              | string_parser)
print(chain_time.invoke({'programming_language': 'java'}))

The time it would take to complete the literature and the projects can vary greatly depending on several factors such as your reading speed, comprehension level, coding speed, familiarity with the topics, and the complexity of the projects. However, here's a rough estimate:

1. "Effective Java" by Joshua Bloch: This book has around 400 pages. If you read and practice for about 2 hours a day, it might take you around 2-3 weeks.

2. "Java: The Complete Reference" by Herbert Schildt: This is a comprehensive guide with over 1000 pages. If you spend 2 hours a day, it might take you around 2-3 months.

3. "Java Concurrency in Practice" by Brian Goetz: This book has around 400 pages. If you read and practice for about 2 hours a day, it might take you around 2-3 weeks.

For the projects:

1. Building a Weather Application using OpenWeatherMap API: If you're familiar with APIs, it might take you around 1-2 weeks.

2. Developing a Simple E-commerce Website with Shopping Cart: This is a more comp

In [27]:
chain_time2.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            